In [1]:
from IPython.display import display, HTML

display(
    HTML("""
<style>
div.container{width:80% !important;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
span.toc-item-num{display:none;}
div.CodeMirror {font-family:Consolas}
div.input {font-family:Consolas}
</style>
"""))

**<font size='5' color='red'>ch03. 연관분석</font>**
- pip install apyori

# 1. 연관분석 개요
 - 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느 정도인지 분석하는 방법
 - 활용분야 : 상품 진열, 사기 보험 적발, 신상품 카테고리 구성, ...
 
```
{조건:X ex 오렌지주스} -> {결과:Y ex 와인}
연관분석과 관련된 지표
1. 지지도(support) : 전체 데이터 중 조건결과 항목들이 포함된 거래 비율
    (X, Y)의 항목수 / 전체 데이터 수 ex.0.2
2. 신뢰도(confidence) : 조건이 발생했을 때, 결과가 동시에 일어날 확률
    (X, Y)의 항목수 / 조건항목수 ex.0.5
3. 향상도(lift) : 우연히 발생한 규칙인지를 확인
    조건결과 지지도 / (조건지지도)*(결과지지도)
    < 1 : 음의 상관관계
    > 1 : 양의 상관관계
    
                        지지도     신뢰도       향상도
{오렌지주스} -> {와인}   1/5(0.2)  1/2(0.5)  0.2/0.4*0.6(0.2/0.24) = 0.833
```

In [4]:
%ls data\cf_basket.csv

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: EC0C-568F

 D:\imsehyeon\source\10_자연어처리\data 디렉터리

2024-12-30  오전 09:03               119 cf_basket.csv
               1개 파일                 119 바이트
               0개 디렉터리  231,819,591,680 바이트 남음


# 2. 연관분석 구현

In [8]:
# 트랜잭션 데이터 가져오기
import csv
transaction = []
with open('data/cf_basket.csv', 'r', encoding='utf-8') as f:
    csvdata = csv.reader(f)
    # print(list(csvdata))
    for row in csvdata:
        transaction.append(row)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [14]:
from apyori import apriori
rules = apriori(transaction, min_support=0.15, min_confidence=0.1)
rules = list(rules)
len(rules)

18

In [17]:
rules[15]
# items=frozenset({'맥주', '콜라', '와인'}), support=0.2 : 이 세트가 함께 나타날 확률
# items_base : 조건(lhs)
# items_add : 결과(rhs)

RelationRecord(items=frozenset({'맥주', '콜라', '와인'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주', '콜라', '와인'}), confidence=0.2, lift=1.0), OrderedStatistic(items_base=frozenset({'맥주'}), items_add=frozenset({'콜라', '와인'}), confidence=0.5, lift=1.25), OrderedStatistic(items_base=frozenset({'와인'}), items_add=frozenset({'맥주', '콜라'}), confidence=0.33333333333333337, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'맥주', '와인'}), confidence=0.25, lift=1.25), OrderedStatistic(items_base=frozenset({'맥주', '와인'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'맥주', '콜라'}), items_add=frozenset({'와인'}), confidence=0.5, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라', '와인'}), items_add=frozenset({'맥주'}), confidence=0.5, lift=1.25)])

In [21]:
row = rules[15]
support = row[1]
ordered_st = row[2]
for item in ordered_st:
    lhs = ', '.join([x for x in item[0]])
    rhs = ', '.join([x for x in item[1]])
    confidence = item[2]
    lift       = item[3]
    if lift > 1:
        print('{} => {} \t {} \t {} \t {}'.format(lhs, rhs, support, confidence, lift))

맥주 => 콜라, 와인 	 0.2 	 0.5 	 1.25
콜라 => 맥주, 와인 	 0.2 	 0.25 	 1.25
맥주, 와인 => 콜라 	 0.2 	 1.0 	 1.25
콜라, 와인 => 맥주 	 0.2 	 0.5 	 1.25


In [37]:
print('조건 => 결과 \t 지지도(support) \t 신뢰도(confidence) \t 향상도(lift)')
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift       = item[3]
        if lift > 1:
            print('{} => {} : {:10} \t {:15} \t {:15}'.format(lhs, rhs, 
                                                       support, 
                                                       round(confidence, 2), 
                                                       round(lift, 2)))

조건 => 결과 	 지지도(support) 	 신뢰도(confidence) 	 향상도(lift)
맥주 => 콜라 :        0.4 	             1.0 	            1.25
콜라 => 맥주 :        0.4 	             0.5 	            1.25
소주 => 콜라 :        0.6 	             1.0 	            1.25
콜라 => 소주 :        0.6 	            0.75 	            1.25
콜라 => 맥주, 소주 :        0.2 	            0.25 	            1.25
맥주, 소주 => 콜라 :        0.2 	             1.0 	            1.25
맥주 => 콜라, 와인 :        0.2 	             0.5 	            1.25
콜라 => 맥주, 와인 :        0.2 	            0.25 	            1.25
맥주, 와인 => 콜라 :        0.2 	             1.0 	            1.25
콜라, 와인 => 맥주 :        0.2 	             0.5 	            1.25
소주 => 콜라, 오렌지주스 :        0.2 	            0.33 	            1.67
콜라 => 소주, 오렌지주스 :        0.2 	            0.25 	            1.25
소주, 오렌지주스 => 콜라 :        0.2 	             1.0 	            1.25
콜라, 오렌지주스 => 소주 :        0.2 	             1.0 	            1.67
콜라 => 소주, 와인 :        0.2 	            0.25 	            1.25
소주, 와인 => 콜라 :      

In [42]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs', 'rhs', 'support(지지도)', 'confidence(신뢰도)', 'lift(향상도)'])

index = 0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift       = item[3]
        if lift > 1:
            rules_df.loc[index] = [lhs, rhs, support, round(confidence, 2), round(lift, 2)]
            index += 1
rules_df.sort_values(by=['confidence(신뢰도)', 'lift(향상도)'], ascending=False)

,lhs,rhs,support(지지도),confidence(신뢰도),lift(향상도)
13,"콜라, 오렌지주스",소주,0.2,1.00,1.67
0,맥주,콜라,0.4,1.00,1.25
2,소주,콜라,0.6,1.00,1.25
5,"맥주, 소주",콜라,0.2,1.00,1.25
8,"맥주, 와인",콜라,0.2,1.00,1.25
12,"소주, 오렌지주스",콜라,0.2,1.00,1.25
15,"소주, 와인",콜라,0.2,1.00,1.25
3,콜라,소주,0.6,0.75,1.25
1,콜라,맥주,0.4,0.50,1.25
6,맥주,"콜라, 와인",0.2,0.50,1.25


# 3. 뉴스 연관 분석
 - https://fs.jtbc.co.kr/RSS/economy.xml 기사 검색 후 -> 명사 추출(기자, 앵커 제외)
     -> 연관분석 (min_support=0.15, min_confidence=0.1)

In [64]:
import requests
from bs4 import BeautifulSoup
from mecab import MeCab

rss_url = 'http://fs.jtbc.joins.com/RSS/economy.xml'

In [57]:
stopwords = ['기자', '앵커']
noun_list = ['홍수', '기자', '앵커', '단군']
list(filter(lambda word : word not in stopwords, noun_list))

['홍수', '단군']

In [100]:
jtbc_economy = requests.get(rss_url)
soup = BeautifulSoup(jtbc_economy.content, 'xml')
link_ele = soup.select('item link')
link_list = [link.text for link in link_ele]
stopwords = ('기자', '앵커')
mecab = MeCab()
news = []
for link in link_list:
    response = requests.get(link)
    news_soup = BeautifulSoup(response.content, 'html.parser')
    title = news_soup.select_one('title').text.replace(' | JTBC 뉴스', '')
    content = str(news_soup.select_one('meta[name="description"]'))
    content = content.replace('<meta content="', '').replace('" name="description"/>', '')
    # 일반명사:NNG, 고유명사:NNP
    noun_list = mecab.nouns(title + ' ' + content)
    # noun_list = [word for word, tag in komoran.pos(title + ' ' + content) if tag in ('NNG', 'NNP')]  # 위와 둘 중 하나
    # noun_list = [word for word, tag in mecab.pos(title + ' ' + content) if (tag in ('NNG', 'NNP')) & (word not in ('기자', '앵커'))]
    noun_list = list(filter(lambda word : word not in stopwords, noun_list))  # 미사용 시 바로 위 소스만 사용
    news.append(noun_list)
news

[['둔촌',
  '주공',
  '전세',
  '홍수',
  '예상',
  '눈치',
  '게임',
  '시작',
  '단군',
  '이래',
  '최대',
  '규모',
  '재건축',
  '둔촌',
  '주공',
  '아파트',
  '입주',
  '다음',
  '달',
  '말',
  '시작',
  '규모',
  '입주',
  '전세',
  '매물',
  '현장',
  '분위기',
  '이상',
  '다음',
  '달',
  '말',
  '만',
  '천',
  '세대',
  '입주',
  '시작',
  '서울',
  '둔촌동',
  '올림픽',
  '파크',
  '포레',
  '온',
  '국내',
  '건축',
  '가운데',
  '역대',
  '최대',
  '규모',
  '이용택',
  '공인',
  '중개사',
  '공개',
  '행사',
  '이후',
  '분',
  '배',
  '가량',
  '판단',
  '연령',
  '세대',
  '관심',
  '단지',
  '입주',
  '시기',
  '전',
  '전세',
  '물량',
  '지도',
  '관심',
  '년',
  '전',
  '천',
  '세대',
  '입주',
  '서울',
  '헬리오',
  '시티',
  '당시',
  '전세',
  '매매',
  '가격',
  '동반',
  '하락',
  '이번',
  '게',
  '현장',
  '분위깁니다',
  '김복수',
  '공인',
  '중개사',
  '일반',
  '아파트',
  '입주',
  '정도',
  '전세',
  '게',
  '보통',
  '시장',
  '모습',
  '여기',
  '전세',
  '입주',
  '전세',
  '본인',
  '가격',
  '거',
  '거주',
  '의무',
  '폐지',
  '데',
  '신축',
  '선호',
  '현상',
  '가세',
  '전세',
  '매물',
  '자체',
  '부족',
  '겁니다',
  '인터넷',
  '매물',
  '천',
  '세대',
  '최근',
  '정부',
 

In [95]:
from gensim.models import Word2Vec
model = Word2Vec(news, 
                 window=10,    # 각 단어 좌우 10개 단어를 학습 컨텍스트로 사용
                 min_count=2,  # 훈련 데이터에서 2회 이상 등장한 단어만 학습
                 workers=-1)   # 병렬 처리 수. -1:모든 가용 코어 사용

In [96]:
from apyori import apriori
rules = apriori(news, min_support=0.15, min_confidence=0.1)
rules = list(rules)
len(rules)

import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs', 'rhs', 'support(지지도)', 'confidence(신뢰도)', 'lift(향상도)'])

index = 0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift       = item[3]
        if lift > 1:
            rules_df.loc[index] = [lhs, rhs, support, round(confidence, 2), round(lift, 2)]
            index += 1
rules_df.sort_values(by=['confidence(신뢰도)', 'lift(향상도)'], ascending=False)

,lhs,rhs,support(지지도),confidence(신뢰도),lift(향상도)
6,사실,상태,0.15,1.00,6.67
7,상태,사실,0.15,1.00,6.67
8,사실,업체,0.15,1.00,6.67
9,업체,사실,0.15,1.00,6.67
10,상태,업체,0.15,1.00,6.67
11,업체,상태,0.15,1.00,6.67
12,사실,"상태, 업체",0.15,1.00,6.67
13,상태,"사실, 업체",0.15,1.00,6.67
14,업체,"상태, 사실",0.15,1.00,6.67
15,"상태, 사실",업체,0.15,1.00,6.67
